In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## EDA Inicial

In [ ]:
# Ruta del archivo
ruta = r"C:\Users\marta\Desktop\mibootcamp\ETL_IBIShotels\data\reservas_hoteles.parquet"

# Leer el archivo Parquet con pandas
df_raw = pd.read_parquet(ruta, engine="pyarrow")

# Ignorar el índice del DataFrame
df_raw = df_raw.reset_index(drop=True)

In [3]:
df = df_raw.copy()
df = pd.DataFrame(df)
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
1929,afb9a3e9-849e-463e-8080-c2a470f59c68,87a7ef0e-35a6-407c-b8a8-39ce6322d9fe,Eleuterio,Gabaldón,eleuterio.gabaldón@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,414.01,Hotel Los Almendros,5.0,Madrid
5482,78790b1a-69e0-4688-92f4-f8be1af13a61,328da559-3fb3-47ed-bf7d-515fe6d18647,Jesús,Briones,jesús.briones@example.com,False,2025-02-08,2025-03-01,2025-03-02,48,111.32,Hotel Vista Alegre,3.0,Madrid
15077,b0c61f74-97f1-4ab1-9e0a-b8f1cde2b626,48f444c6-2c13-4636-bef5-37acb0de4d3b,Eutropio,Bonilla,eutropio.bonilla@example.com,False,2025-02-12,2025-03-01,2025-03-02,29,444.27,Hotel Las Estrellas,2.0,Madrid
7307,6a3ce8b1-6537-4bad-9c8d-c0b4f6e37ea6,53ee02da-59d1-4325-84c8-4e676e2f431b,Agustín,Perelló,agustín.perelló@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,361.35,Hotel Maravilla Real,3.0,Madrid
5574,1362d1d8-7c24-4cee-a6c2-35c8aa457841,c401c5a1-4f22-436e-8b80-47a9db028a68,Gloria,Batalla,gloria.batalla@example.com,True,,2025-03-01,2025-03-02,135,NaN,,NaN,


In [59]:
df.shape

(15000, 14)

In [4]:
df_col_fecha = df[["fecha_reserva", "inicio_estancia", "final_estancia"]]
df_col_fecha

,fecha_reserva,inicio_estancia,final_estancia
0,,2025-03-01,2025-03-02
1,,2025-03-01,2025-03-02
2,2025-02-09,2025-03-01,2025-03-02
3,2025-02-08,2025-03-01,2025-03-02
4,2025-02-02,2025-03-01,2025-03-02
...,...,...,...
15093,2025-02-02,2025-03-01,2025-03-02
15094,2025-02-07,2025-03-01,2025-03-02
15095,2025-02-08,2025-03-01,2025-03-02
15096,2025-02-10,2025-03-01,2025-03-02


## Transformación inicial 

- transformación de co.fecha_reserva, incio_estancia, final_etancia check
- hay nulos en las columnas: inicio_estancia, final_instancia, precio_noche, nombre_hotel, estrellas.
- analizar duplicados
- asegurarse del tipo de dato es el correcto

In [5]:
def convertir_a_fecha(lista_columnas, dataframe):
    for col in lista_columnas:
        dataframe[col] = pd.to_datetime(dataframe[col], errors='coerce')
    return dataframe

In [6]:
col_fechas = ['inicio_estancia', 'final_estancia', 'fecha_reserva']
df = convertir_a_fecha(col_fechas, df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

In [7]:
def convertir_fecha(dataframe):
    for col in dataframe.filter(like="fecha", axis=1):
        dataframe[col] = pd.to_datetime(dataframe[col], errors='coerce')  # Forzar conversión
    return dataframe

# Aplicar la conversión a todas las columnas que contengan "fecha"
df = convertir_fecha(df)

# Verificar tipos de datos
df.info()                                    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

## Incosistencia en los datos:

In [8]:
df["nombre_hotel"].nunique()

20

In [9]:
df["id_hotel"].nunique()

29

In [10]:
df['id_cliente'].nunique()


14847

In [11]:
df["id_reserva"].nunique()

15000

- debería haber los mismos nombres de hoteles que de id_hotel: hay más id_hotel que nombre:hotel por lo que puede haber un error al asignar los ID
- debería hacer los mismos id_clientes que id_reserva: hay más reservas que clientes, por lo que hay clientes que han hecho más de una reserva lo que en nuestro contexto no tiene sentido ya que los datos son para un mismo fin de semana. 

2️⃣ Analizando clientes con más de una reserva

In [12]:
df_clientes_check = df.groupby("id_cliente")["id_reserva"].count().reset_index()
df_clientes_check = df_clientes_check.sort_values(by="id_reserva", ascending=False)
print(df_clientes_check.head(10))  # Ver los 10 clientes con más reservas

                                 id_cliente  id_reserva
2080   2407853d-23ff-4b7d-abcc-5c619d8c960d           3
6584   7237d336-8fa8-4ab3-b454-0d969f0ecb84           3
14608  fba96362-d117-4f45-b405-f735d813833a           3
8076   8bec1ff1-84d7-4cad-937f-dbae37691569           3
13957  f0c7fbf6-cea3-42fc-a94f-11c69dbf5713           3
10811  ba60cff3-280a-4948-b6da-5e76e57b9d79           3
8430   91cf76ea-170c-459c-82a1-a2dde074ad38           3
2981   3369b2b1-6b9a-4914-b7bb-655dd1d9c90f           3
7024   79ff038f-5dc8-4b72-956b-0e7989d44a06           2
12736  db7b4be7-5857-42c6-b0f0-aec8b5e3ae9a           2


3️⃣ Analizando la relación entre fechas y reservas

Análisis de duplicados

In [13]:
# Verificar duplicados en el DataFrame
duplicados = df.duplicated()
print(f'Número de filas duplicadas: {duplicados.sum()}')

# Mostrar las filas duplicadas
df[duplicados]

Número de filas duplicadas: 98


,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
15000,9894646a-716a-4a9d-b504-6d05cff706d3,659980f2-c213-4eed-bddf-1bfebc8dc985,Marianela,Pablo,marianela.pablo@example.com,False,2025-02-09,2025-03-01,2025-03-02,29,246.07,Hotel Puerta del Cielo,5.0,Madrid
15001,9754807a-8685-4198-b843-6dc991377377,a8b5e355-d76c-4836-af7d-57021d03228b,Victor,Santamaria,victor.santamaria@example.com,False,2025-02-07,2025-03-01,2025-03-02,49,169.14,Hotel Los Almendros,5.0,Madrid
15002,2a5ea0d1-9584-4458-8a08-e276d7cbbc3a,0f0150df-7805-41dd-84c5-5aadd6816086,Ámbar,Benito,ámbar.benito@example.com,False,2025-02-02,2025-03-01,2025-03-02,19,198.87,Hotel Palacio Imperial,4.0,Madrid
15003,0ba826ae-751c-4cd1-9147-b4c36dd4fc72,c91e350d-1193-41de-834a-1940955e91b6,Tito,Amorós,tito.amorós@example.com,False,2025-02-12,2025-03-01,2025-03-02,14,54.32,Hotel Las Estrellas,1.0,Madrid
15004,1e5d7ead-36e9-4b22-b870-9a122355b763,4ca7383c-3e0c-4403-b0dc-f2a022edb524,Gabriel,Cruz,gabriel.cruz@example.com,False,2025-02-10,2025-03-01,2025-03-02,19,405.23,Hotel Jardines del Rey,5.0,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid


In [14]:
### limpiamos los duplicados que coinciden en todos sus valores
df = df.drop_duplicates()

Gestión de nulos

4. Validación de Unicidad y Claves Primarias

In [15]:
df['id_reserva'].nunique() == df.shape[0]  # Debe ser True si todos son únicos

True

In [16]:
df["mail"].nunique(), df["id_cliente"].nunique()


(14905, 14847)

## Correción ID_clientes únicos


In [17]:
df["mail"].nunique()

14905

In [18]:
df.groupby("mail")["id_cliente"].nunique().sort_values(ascending=False)


mail
ariadna.miguel@example.com        3
victoriano.ródenas@example.com    3
victor.peiró@example.com          2
manuelita.leiva@example.com       2
aitor.prieto@example.com          2
                                 ..
eustaquio.oliveras@example.com    1
eustaquio.paz@example.com         1
eustaquio.portillo@example.com    1
eustaquio.quintero@example.com    1
úrsula.ureña@example.com          1
Name: id_cliente, Length: 14905, dtype: int64

In [19]:
# Agrupar por 'mail', contar valores únicos de 'id_cliente', y filtrar los que tienen más de 1 valor único
resultado = df.groupby("mail")["id_cliente"].nunique()
resultado_filtrado = resultado[resultado > 1].sort_values(ascending=False)

cantidad_emails = len(resultado_filtrado)

print(f'La cantidad de mail que tienen más de un valour único en id_cliente es {cantidad_emails}')

La cantidad de mail que tienen más de un valour único en id_cliente es 93


In [20]:
df["mail"].nunique()

14905

In [21]:
df["id_cliente"].nunique()

14847

In [22]:
df["id_cliente"] = df["mail"].factorize()[0]

In [23]:
df.groupby("id_cliente")["mail"].nunique().max()


1

In [60]:
df["id_cliente"].nunique()

14905

## Corrección ID_Hoteles
- aqui es donde vas a tener q dividir la base de datos de los hoteles de la competencia y los mios. 
- crear un nuevo id_hotel nuevo atraves del nombre del hotel : problema: los nombres de la competencia no los tenemos. 


- Analizando inconsistencias en ID de hoteles y nombres

In [24]:
df_hotel_check = df.groupby("nombre_hotel")["id_hotel"].nunique().reset_index()
df_hotel_check

,nombre_hotel,id_hotel
0,,10
1,Gran Hotel Madrid,19
2,Hotel Brisas del Mar,19
3,Hotel Camino del Sol,19
4,Hotel Costa Azul,19
5,Hotel Encanto Real,19
6,Hotel Jardines del Rey,19
7,Hotel Las Estrellas,19
8,Hotel Los Almendros,19
9,Hotel Luz de Madrid,19


In [25]:
prop = df[df["competencia"] == False].copy()
comp = df[df["competencia"] == True].copy()

## Con el que por fin me ha salido:

In [26]:
# Asignar un valor único de id_hotel a cada nombre único de hotel
prop["id_hotel"] = pd.factorize(prop["nombre_hotel"])[0] + 1

# Mostrar el DataFrame con los id_hotel asignados
print(prop[["nombre_hotel", "id_hotel"]].drop_duplicates().sort_values(by="id_hotel"))

              nombre_hotel  id_hotel
2        Hotel Monte Verde         1
3     Hotel Brisas del Mar         2
4     Hotel Camino del Sol         3
5   Hotel Puerta del Cielo         4
6       Hotel Encanto Real         5
8          Palacio del Sol         6
10  Hotel Jardines del Rey         7
13     Hotel Las Estrellas         8
15       Gran Hotel Madrid         9
17      Hotel Torre Dorada        10
18  Hotel Palacio Imperial        11
20     Hotel Luz de Madrid        12
22     Hotel Los Almendros        13
24        Hotel Sol y Luna        14
26      Hotel Mirador Real        15
32     Hotel Rincón Sereno        16
33      Hotel Vista Alegre        17
37        Hotel Costa Azul        18
60    Hotel Maravilla Real        19


In [27]:
prop["id_hotel"].nunique()

19

In [28]:
print("Número de nombres de hoteles únicos:", prop["nombre_hotel"].nunique())
print("Número de id_hotel únicos:", prop["id_hotel"].nunique())


Número de nombres de hoteles únicos: 19
Número de id_hotel únicos: 19


## Competencia

In [29]:
print("Número de nombres de hoteles únicos:", comp["nombre_hotel"].nunique())
print("Número de id_hotel únicos:", comp["id_hotel"].nunique())

Número de nombres de hoteles únicos: 1
Número de id_hotel únicos: 10


In [30]:
comp

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,0,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,1,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,
7,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,7,Judith,Rivas,judith.rivas@example.com,True,NaT,2025-03-01,2025-03-02,131,NaN,,NaN,
9,458029ad-cc9b-4d85-8290-fec170bcc02d,9,Adolfo,Cuervo,adolfo.cuervo@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,
11,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,11,María Carmen,Navarro,maría carmen.navarro@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14989,e856adee-838c-4031-920c-05ec5f1e32e6,14894,Jesusa,Espinosa,jesusa.espinosa@example.com,True,NaT,2025-03-01,2025-03-02,135,NaN,,NaN,
14991,77134049-6e4f-49b0-a6f8-93b7466b1c22,14896,Cruz,Hurtado,cruz.hurtado@example.com,True,NaT,2025-03-01,2025-03-02,128,NaN,,NaN,
14993,19aef6f3-3e85-4719-9dff-b768d9bfe5a9,14898,Jose Francisco,Cortes,jose francisco.cortes@example.com,True,NaT,2025-03-01,2025-03-02,186,NaN,,NaN,
14996,aa092c3e-7fe6-4985-a5a8-cacd38554b1b,14901,Toño,Narváez,toño.narváez@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,


In [31]:
df_hot = pd.read_csv(r"C:\Users\marta\Desktop\mibootcamp\ETL_IBIShotels\data\df_hot.csv")
df_hot

,nombre,estrellas,precio
0,ibis Styles Madrid Prado,4.7,170
1,Novotel Madrid Center,4.6,255
2,ibis budget Madrid Calle 30,4.4,106
3,ibis Madrid Centro las Ventas,4.5,172
4,Novotel Madrid City Las Ventas,4.6,173
5,ibis budget Madrid Vallecas,4.3,102
6,Pullman Madrid Airport & Feria,4.2,127
7,ibis Madrid Aeropuerto Barajas,4.4,116
8,ibis Madrid Alcorcon Tresaguas,4.4,90
9,ibis budget Madrid Aeropuerto,4.0,97


In [32]:
df_hot.rename(columns={"nombre": "nombre_hotel", "estrellas": "estrellas", "precio": "precio_noche"}, inplace=True)

In [33]:
comp["id_hotel"].unique()

array([113, 194, 131, 114, 103, 181, 128, 186, 135, 117], dtype=int64)

In [34]:
# Obtener los valores únicos de id_hotel de comp y convertirlos a una lista
id_hoteles_unicos = comp["id_hotel"].unique().tolist()

# Mostrar la lista de valores únicos
print(id_hoteles_unicos)

[113, 194, 131, 114, 103, 181, 128, 186, 135, 117]


In [ ]:
# anañadi la columna id_hotel a df_hot con los valores unidocs de id_hotel de comp
df_hot["id_hotel"] = id_hoteles_unicos

In [37]:
df_hot

,nombre_hotel,estrellas,precio_noche,id_hotel
0,ibis Styles Madrid Prado,4.7,170,113
1,Novotel Madrid Center,4.6,255,194
2,ibis budget Madrid Calle 30,4.4,106,131
3,ibis Madrid Centro las Ventas,4.5,172,114
4,Novotel Madrid City Las Ventas,4.6,173,103
5,ibis budget Madrid Vallecas,4.3,102,181
6,Pullman Madrid Airport & Feria,4.2,127,128
7,ibis Madrid Aeropuerto Barajas,4.4,116,186
8,ibis Madrid Alcorcon Tresaguas,4.4,90,135
9,ibis budget Madrid Aeropuerto,4.0,97,117


In [67]:
df_hot.dtypes

nombre_hotel     object
estrellas       float64
precio_noche      int64
id_hotel          int64
dtype: object

In [38]:
dfill = comp.merge(df_hot, on='id_hotel', how='left')
dfill = dfill.drop(columns=["nombre_hotel_x", "estrellas_x", "precio_noche_x"])
dfill.rename(columns={"nombre_hotel_y": "nombre_hotel", "estrellas_y": "valoración", "precio_noche_y": "precio_noche"}, inplace=True)

# Mostrar el DataFrame resultante
dfill

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche_x,nombre_hotel_x,estrellas_x,ciudad,nombre_hotel_y,estrellas_y,precio_noche_y
0,5256cc90-139b-43d2-8ec5-412495d751cf,0,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,ibis Styles Madrid Prado,4.7,170
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,1,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,Novotel Madrid Center,4.6,255
2,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,7,Judith,Rivas,judith.rivas@example.com,True,NaT,2025-03-01,2025-03-02,131,NaN,,NaN,,ibis budget Madrid Calle 30,4.4,106
3,458029ad-cc9b-4d85-8290-fec170bcc02d,9,Adolfo,Cuervo,adolfo.cuervo@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,,ibis Madrid Centro las Ventas,4.5,172
4,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,11,María Carmen,Navarro,maría carmen.navarro@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,,Novotel Madrid City Las Ventas,4.6,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,e856adee-838c-4031-920c-05ec5f1e32e6,14894,Jesusa,Espinosa,jesusa.espinosa@example.com,True,NaT,2025-03-01,2025-03-02,135,NaN,,NaN,,ibis Madrid Alcorcon Tresaguas,4.4,90
5168,77134049-6e4f-49b0-a6f8-93b7466b1c22,14896,Cruz,Hurtado,cruz.hurtado@example.com,True,NaT,2025-03-01,2025-03-02,128,NaN,,NaN,,Pullman Madrid Airport & Feria,4.2,127
5169,19aef6f3-3e85-4719-9dff-b768d9bfe5a9,14898,Jose Francisco,Cortes,jose francisco.cortes@example.com,True,NaT,2025-03-01,2025-03-02,186,NaN,,NaN,,ibis Madrid Aeropuerto Barajas,4.4,116
5170,aa092c3e-7fe6-4985-a5a8-cacd38554b1b,14901,Toño,Narváez,toño.narváez@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,,ibis Madrid Centro las Ventas,4.5,172


## Unión dataset hoteles, reservas completo

In [64]:
dfill["nombre_hotel"].nunique()


10

In [65]:
dfill["id_hotel"].nunique()

10

In [50]:
df_concat = pd.concat([prop, dfill], axis=0, ignore_index=True)

In [66]:
df_concat["id_cliente"].nunique()

14905

In [80]:
df.shape

(15000, 14)

In [68]:
df_concat["nombre_hotel"].nunique()

29

## Últimas transformaciones en el df: normalización, nulos etc.

### Columna Nombre_Hotel

In [125]:
df_concat["nombre_hotel"].value_counts()

nombre_hotel
Hotel Sol y Luna                  557
Hotel Monte Verde                 550
ibis Madrid Centro las Ventas     537
Hotel Luz de Madrid               536
Hotel Brisas del Mar              534
Novotel Madrid City Las Ventas    533
Hotel Los Almendros               533
Hotel Mirador Real                533
Pullman Madrid Airport & Feria    530
Hotel Puerta del Cielo            526
ibis budget Madrid Calle 30       524
ibis Madrid Aeropuerto Barajas    522
Hotel Camino del Sol              518
Gran Hotel Madrid                 518
ibis Madrid Alcorcon Tresaguas    518
ibis Styles Madrid Prado          515
Hotel Costa Azul                  515
Hotel Las Estrellas               514
Hotel Rincón Sereno               512
Hotel Torre Dorada                512
Hotel Encanto Real                511
Hotel Vista Alegre                511
Novotel Madrid Center             511
Palacio del Sol                   510
Hotel Jardines del Rey            502
ibis budget Madrid Vallecas       493

In [134]:
df_concat['nombre_hotel'] = df_concat['nombre_hotel'].str.title()

# Mostrar el DataFrame resultante
print(df_concat[['nombre_hotel']].sample(5))

                nombre_hotel
340      Hotel Los Almendros
8766      Hotel Vista Alegre
5304  Hotel Jardines Del Rey
4173      Hotel Torre Dorada
9680    Hotel Maravilla Real


In [96]:
df_concat.dtypes

id_reserva                 object
id_cliente                  int64
nombre                     object
apellido                   object
mail                       object
competencia                  bool
fecha_reserva      datetime64[ns]
inicio_estancia    datetime64[ns]
final_estancia     datetime64[ns]
id_hotel                    int64
precio_noche              float64
nombre_hotel               object
valoración                float64
ciudad                     object
dtype: object

In [ ]:
# renombrar columna
df_concat.rename(columns={"estrellas": "valoración"}, inplace=True)

In [93]:
df_concat.drop(columns=["valoracion"], inplace=True)

## Comprobación email


In [70]:
df_concat["id_cliente"].nunique()

14905

In [78]:
df_concat["mail"].count()

15000

In [73]:
df_concat["mail"].str.contains('@').sum()

15000

In [ ]:
##df_concat.to_csv(r"C:\Users\marta\Desktop\mibootcamp\ETL_IBIShotels\data\dfcompleto.csv", index=False)

## Promedio de estrellas

In [103]:
df_concat["nombre_hotel"].value_counts()

nombre_hotel
Hotel Sol y Luna                  557
Hotel Monte Verde                 550
ibis Madrid Centro las Ventas     537
Hotel Luz de Madrid               536
Hotel Brisas del Mar              534
Novotel Madrid City Las Ventas    533
Hotel Los Almendros               533
Hotel Mirador Real                533
Pullman Madrid Airport & Feria    530
Hotel Puerta del Cielo            526
ibis budget Madrid Calle 30       524
ibis Madrid Aeropuerto Barajas    522
Hotel Camino del Sol              518
Gran Hotel Madrid                 518
ibis Madrid Alcorcon Tresaguas    518
ibis Styles Madrid Prado          515
Hotel Costa Azul                  515
Hotel Las Estrellas               514
Hotel Rincón Sereno               512
Hotel Torre Dorada                512
Hotel Encanto Real                511
Hotel Vista Alegre                511
Novotel Madrid Center             511
Palacio del Sol                   510
Hotel Jardines del Rey            502
ibis budget Madrid Vallecas       493

In [112]:
df_concat["valoración"].isna().sum()

5172

In [113]:
 # Calcular la valoración promedio para cada hotel
valoracion_promedio = df_concat.groupby('nombre_hotel')['valoración'].mean()

In [121]:
valoracion_promedio

nombre_hotel
Gran Hotel Madrid                 3.067568
Hotel Brisas del Mar              3.088015
Hotel Camino del Sol              3.067568
Hotel Costa Azul                  3.118447
Hotel Encanto Real                3.031311
Hotel Jardines del Rey            2.936255
Hotel Las Estrellas               2.906615
Hotel Los Almendros               3.011257
Hotel Luz de Madrid               3.052239
Hotel Maravilla Real              2.978858
Hotel Mirador Real                2.977486
Hotel Monte Verde                 3.103636
Hotel Palacio Imperial            3.006479
Hotel Puerta del Cielo            3.028517
Hotel Rincón Sereno               2.998047
Hotel Sol y Luna                  3.008977
Hotel Torre Dorada                2.919922
Hotel Vista Alegre                2.941292
Novotel Madrid Center                  NaN
Novotel Madrid City Las Ventas         NaN
Palacio del Sol                   2.992157
Pullman Madrid Airport & Feria         NaN
ibis Madrid Aeropuerto Barajas         Na

In [ ]:
## esto no es df_concat['valoración'] = df_concat.apply(
    lambda row: valoracion_promedio[row['nombre_hotel']] if pd.isna(row['valoración']) else row['valoración'],
    axis=1
)


In [117]:
# Contar el número de valoraciones por hotel
conteo_valoraciones = df_concat.groupby("nombre_hotel")["valoración"].count()
print("Número de valoraciones por hotel:")
print(conteo_valoraciones)

Número de valoraciones por hotel:
nombre_hotel
Gran Hotel Madrid                 518
Hotel Brisas del Mar              534
Hotel Camino del Sol              518
Hotel Costa Azul                  515
Hotel Encanto Real                511
Hotel Jardines del Rey            502
Hotel Las Estrellas               514
Hotel Los Almendros               533
Hotel Luz de Madrid               536
Hotel Maravilla Real              473
Hotel Mirador Real                533
Hotel Monte Verde                 550
Hotel Palacio Imperial            463
Hotel Puerta del Cielo            526
Hotel Rincón Sereno               512
Hotel Sol y Luna                  557
Hotel Torre Dorada                512
Hotel Vista Alegre                511
Novotel Madrid Center               0
Novotel Madrid City Las Ventas      0
Palacio del Sol                   510
Pullman Madrid Airport & Feria      0
ibis Madrid Aeropuerto Barajas      0
ibis Madrid Alcorcon Tresaguas      0
ibis Madrid Centro las Ventas       0
ibi

In [120]:
# Filtrar los datos para incluir solo las filas donde la valoración es NaN
valoraciones_nulas = df_concat[df_concat["valoración"].isna()]

# Contar el número de valoraciones nulas por hotel
conteo_valoraciones_nulas = valoraciones_nulas.groupby("nombre_hotel")["valoración"].count()

# Mostrar el número de valoraciones nulas por hotel
print("Número de valoraciones nulas por hotel:")
print(conteo_valoraciones_nulas)

Número de valoraciones nulas por hotel:
nombre_hotel
Novotel Madrid Center             0
Novotel Madrid City Las Ventas    0
Pullman Madrid Airport & Feria    0
ibis Madrid Aeropuerto Barajas    0
ibis Madrid Alcorcon Tresaguas    0
ibis Madrid Centro las Ventas     0
ibis Styles Madrid Prado          0
ibis budget Madrid Aeropuerto     0
ibis budget Madrid Calle 30       0
ibis budget Madrid Vallecas       0
Name: valoración, dtype: int64


In [ ]:
# Calcular la media global de todas las valoraciones no nulas
media_global = df_concat['valoración'].mean()

# Rellenar las celdas NaN en la columna 'valoración' con la media global
df_concat['valoración'].fillna(media_global, inplace=True)

In [124]:
df_concat["valoración"].isna().sum()

0

In [135]:
df_concat.to_csv(r"C:\Users\marta\Desktop\mibootcamp\ETL_IBIShotels\data\dfcompleto.csv", index=False)

In [136]:
df_concat["precio_noche"].isna().sum()

52